# ДЗ 3. Логистическая регрессия. Log Loss

In [1]:
import numpy as np

In [2]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [15]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [14]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])
X_st

array([[ 1.        ,  1.        , -0.97958969,  1.        ],
       [ 1.        ,  1.        , -0.56713087,  1.        ],
       [ 1.        ,  2.        , -0.46401617,  2.        ],
       [ 1.        ,  5.        , -0.77336028,  1.        ],
       [ 1.        ,  3.        ,  0.97958969,  2.        ],
       [ 1.        ,  0.        , -0.36090146,  1.        ],
       [ 1.        ,  5.        ,  1.08270439,  3.        ],
       [ 1.        , 10.        ,  2.11385144,  3.        ],
       [ 1.        ,  1.        , -1.08270439,  1.        ],
       [ 1.        ,  2.        ,  0.05155735,  2.        ]])

## * Функция calc_logloss

Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [11]:
def calc_logloss(y, y_pred):
    # ИЗМЕНЕНИЯ:
    pred_log1 = np.array([0 if x == 0 else np.log(x) for x in y_pred])
    pred_log2 = np.array([0 if x == 1 else np.log(1.0 - x) for x in y_pred])
    # ИЗМЕНЕНИЯ
    err = - np.mean(y * pred_log1 + (1.0 - y) * pred_log2)
    return err

In [12]:
# Пример применения - 1
y1 = np.array([1, 0])
y_pred1 = np.array([0.8, 0.1])
calc_logloss(y1, y_pred1)

0.164252033486018

In [13]:
# Пример применения - 2
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

0.05268025782891314

## Функция eval_model

Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [18]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [28]:
W = eval_model(X_st, y, iterations=150000, eta=1e-1)

0 [ 0.45877546 -0.2833519   0.6449505   1.46641523] 1.1785958344356262
15000 [-13.27314919  -1.59497566  -3.18770467  11.17082282] 0.20973531087251968
30000 [-19.03214673  -2.10018205  -4.88462075  15.84722845] 0.1705495036750294
45000 [-23.30909629  -2.48426706  -6.11770014  19.3238589 ] 0.14907093673762203
60000 [-26.7341771   -2.79398809  -7.09424451  22.10455082] 0.13535136053673655
75000 [-29.60686714  -3.05400933  -7.9082326   24.43273384] 0.12572911152419175
90000 [-32.09545327  -3.27872995  -8.61106042  26.44564896] 0.1185257929395436
105000 [-34.30309667  -3.47717591  -9.23364297  28.22753012] 0.1128694352441267
120000 [-36.29716667  -3.65533148  -9.79588528  29.83342132] 0.10826389582486111
135000 [-38.12387906  -3.8173472  -10.31129251  31.30109779] 0.10440644896197586


## Функция calc_pred_proba

Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [33]:
def calc_pred_proba(W, X):
    y_pred = sigmoid(np.dot(W, X.T))
    return y_pred

In [34]:
y_pred_proba = calc_pred_proba(W, X_st)
y_pred_proba

array([3.64543640e-01, 6.65391063e-03, 1.00000000e+00, 7.98591724e-09,
       9.53954960e-01, 3.68000116e-02, 1.00000000e+00, 1.32168724e-02,
       6.35722721e-01, 9.99999959e-01])

## Функция calc_pred

Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [39]:
def calc_pred(W, X, treshold=0.5):
    y_pred = sigmoid(np.dot(W, X.T)) > treshold
    return y_pred.astype(int)

In [40]:
y_pred = calc_pred(W, X_st)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

## * Функция для подсчета Accuracy

Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score

:/

## Могла ли модель переобучиться? Почему?

Да, могла. Так как слишком мало данных.